## Method1

In [1]:
import trimesh
import pyvista as pv
from utils import *

#### Load a file

In [2]:
mesh = pymesh.load_mesh("data/two_spheres.ply")

In [3]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     256 |
+-----------------------------------+---------+
| Number of faces                   |     504 |
+-----------------------------------+---------+
| Number of intersecting face pairs |      64 |
+-----------------------------------+---------+


In [4]:
visualize(mesh, filename="Initial Mesh")

Widget(value='<iframe src="http://localhost:36907/index.html?ui=P_0x7fd9c65576a0_0&reconnect=auto" class="pyvi…

None

In [5]:
visualize_intersection(mesh, filename="Intersections")

/home/kmaeda/anaconda3/envs/pymesh_env/lib/python3.10/site-packages/pyvista/core/pointset.py:1365: PyVistaDeprecationWarning: The current behavior of `pv.PolyData.n_faces` has been deprecated.
                Use `pv.PolyData.n_cells` or `pv.PolyData.n_faces_strict` instead.
                See the documentation in '`pv.PolyData.n_faces` for more information.
  warnings.warn(


Widget(value='<iframe src="http://localhost:43301/index.html?ui=P_0x7f8a5c1fb850_1&reconnect=auto" class="pyvi…

None

#### Get outer hull (to remove inner components)

In [5]:
outer_hull = pymesh.compute_outer_hull(mesh)

In [7]:
self_intersection_stats(outer_hull)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     264 |
+-----------------------------------+---------+
| Number of faces                   |     524 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [8]:
visualize(outer_hull, filename="After compute_outer_hull()")

Widget(value='<iframe src="http://localhost:43301/index.html?ui=P_0x7f8a59deac80_2&reconnect=auto" class="pyvi…

None

#### Define method1


In [6]:
def method1(mesh, with_rounding=True, precision=11, max_iterations=15):
    """
    Resolves self-intersections in a mesh iteratively.

    Parameters:
        mesh: The input mesh object.
        with_rounding (bool): Enables rounding of vertices for stability. Default is True.
        precision (int): Rounding precision level for vertices.
        max_iterations (int): Maximum number of iterations allowed to resolve intersections.

    Returns:
        mesh: The processed mesh with no self-intersections.
    """
    # Initial rounding of vertices
    if (with_rounding):
        mesh = pymesh.form_mesh(
                np.round(mesh.vertices, precision),
                mesh.faces);
    intersecting_faces = pymesh.detect_self_intersection(mesh);

    # Iterative process to resolve self-intersections
    counter = 0;
    while len(intersecting_faces) > 0 and counter < max_iterations:
        if (with_rounding):
            involved_vertices = np.unique(mesh.faces[intersecting_faces].ravel());

            # Round only the involved vertices
            # Suppose precision = 4. Then,
            # [1.234567, 2.345678, 3.456789] <- One vertex example (x, y, z coords)
            # becomes
            # [1.23, 2.35, 3.46]
            vertices_copy = mesh.vertices.copy()  
            vertices_copy[involved_vertices, :] =\
                    np.round(mesh.vertices[involved_vertices, :],
                            precision//2);
        
            mesh = pymesh.form_mesh(vertices_copy, mesh.faces) 

        mesh = pymesh.resolve_self_intersection(mesh, "igl");
        mesh, __ = pymesh.remove_duplicated_faces(mesh, fins_only=True);
        if (with_rounding):
            mesh = pymesh.form_mesh(
                    np.round(mesh.vertices, precision),
                    mesh.faces);
        mesh = intermediate(mesh) # Reload mesh. Otherwise, the next step fails in some cases.
        intersecting_faces = pymesh.detect_self_intersection(mesh);
        print(len(intersecting_faces))
        counter += 1;

    if len(intersecting_faces) > 0:
        logging.warn("Resolving failed: max iteration reached!");

    return mesh

In [7]:
repaired_mesh = method1(outer_hull)

In [8]:
self_intersection_stats(repaired_mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     264 |
+-----------------------------------+---------+
| Number of faces                   |     524 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [9]:
visualize(repaired_mesh, filename="After applying method1")

Widget(value='<iframe src="http://localhost:36907/index.html?ui=P_0x7fd97c932d40_1&reconnect=auto" class="pyvi…

None

In [13]:
evaluation(outer_hull, repaired_mesh)

+-----------------------------------+-----------+-----------+
| Metric                            |    Before |     After |
+===================================+===========+===========+
| Number of vertices                | 264       | 264       |
+-----------------------------------+-----------+-----------+
| Number of faces                   | 524       | 524       |
+-----------------------------------+-----------+-----------+
| Number of intersecting face pairs |   0       |   0       |
+-----------------------------------+-----------+-----------+
| Volume                            |   6.10774 |   6.10774 |
+-----------------------------------+-----------+-----------+
| Area                              |  17.6163  |  17.6163  |
+-----------------------------------+-----------+-----------+
